In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.manage.configuration import get_profile

from aiida.common.datastructures import CalcInfo, CodeInfo
from aiida.common.folders import Folder

from aiida.engine import CalcJob, WorkChain
from aiida.engine import calcfunction, workfunction, submit, run

from aiida.plugins import DataFactory, WorkflowFactory

from aiida.parsers.parser import Parser
from aiida.orm import Code
from aiida.orm import load_code, load_node
from aiida.orm import Str, Dict, Float, Int, Bool
from aiida.orm import QueryBuilder

import aiida

from ase.io.espresso import write_espresso_in
from ase.io.lammpsdata import read_lammps_data
from ase.build import make_supercell
from ase import Atom, Atoms
import ase

from itertools import combinations_with_replacement
import numpy as np
import os
import subprocess
import shutil
import re
from time import sleep
import spglib
import json

from os.path import expanduser

from tools import NodeBank
from tools import wait_for_node_finished
from alamode_aiida.io import load_atoms
from tools.putstructure import PutStructure


In [ ]:
# 型定義
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
g_graph = False  # graphを作るか

# 実行root directoryの定義
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)

# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

# 計算するroot directoryの定義
key = "root_wd"
g_root_wd = nodebank.load_or_dump(key, Str(os.path.join(os.getcwd(), CWD)))
os.makedirs(g_root_wd.value, exist_ok=True)
g_root_wd


In [ ]:
# 構造ファイルの取得
# lammpsから取得する場合の例
# lammpsで最適化しているはずなのでsimulatorもLAMMPSを用いる。

crystal = "Si222"
nodebank.load_or_dump("cyrstal", Str(crystal))

if crystal == "GaN222":
    g_file_format = Str("lammps-data")  # format
    _path = os.path.abspath("lammps_input/GaN/GaN222.lammps")  # file path

    _diag = np.array([2, 2, 2])
    diag = np.diag(_diag)
    g_diag = ArrayData()
    g_diag.set_array('factor', diag)  # supercellをファイルの構造の何倍にするかの定義をする
    nodebank.dump('factor', g_diag)

elif crystal == "Si222":
    g_file_format = Str("lammps-data")
    _path = os.path.abspath("lammps_input/Si222/Si222.lammps")

    _diag = np.array([2, 2, 2])
    diag = np.diag(_diag)
    g_diag = ArrayData()
    g_diag.set_array('factor', diag)  # supercellをファイルの構造の何倍にするかの定義をする
    nodebank.dump('factor', g_diag)

else:
    raise ValueError('unknow crystal.')

g_diag = nodebank.dump("diag", g_diag)
g_structure_org_filename = Str(_path)

nodebank.dump("structure_org_format", g_file_format)

g_raw_structure = load_atoms(
    g_structure_org_filename, g_file_format)
g_raw_structure.get_ase()


In [ ]:
# structureのstandardized structure を factorで構造倍にして、formatでcwdにsaveする。
inputs = {"cwd": g_root_wd, "factor": g_diag, "format": g_file_format,
          "structure": g_raw_structure}
g_primsuper_structures = run(PutStructure, **inputs)


In [ ]:
g_primsuper_structures  # outout


In [ ]:
g_superstructure = g_primsuper_structures["superstructure"]  # StructureData
g_standardizedstructure = g_primsuper_structures["standardizedstructure"]
g_primstructure = g_primsuper_structures["primstructure"]  # StructureData
g_primstructure.get_ase(), g_standardizedstructure.get_ase(), g_superstructure.get_ase()


In [ ]:
if False:
    pk = g_ciffilename.pk
    print(pk)
    !verdi node graph generate $pk


In [ ]:
nodebank.dump("superstructure", g_superstructure)
nodebank.dump("standardizedstructure", g_standardizedstructure)
nodebank.dump("primstructure", g_primstructure)


In [ ]:
raise


In [ ]:
atoms = g_superstructure.get_ase()

In [ ]:
g_cwd = nodebank.load('cwd')
g_cwd

In [ ]:
g_norder = nodebank.load('harmonic_norder')
g_norder

In [ ]:
alm_alm_suggest = WorkflowFactory('alamode.alm_ALM_suggest')


In [ ]:
cutoff_radii = List(list=[[-1]])
cutoff_radii.store()
nbody=List(list=[2])
nbody.store()
inputs = {#"cwd": g_cwd, 
          "structure": g_superstructure,
         "norder": g_norder, "cutoff_radii": cutoff_radii, "nbody": nbody}
ret = submit(alm_alm_suggest,**inputs)
wait_for_node_finished(ret)
ret

In [ ]:
ret.outputs.pattern


In [ ]:
with open("/home/max/Documents/aiida-test/alamode-aiida/example/run31/Ga2N2_2x0x0x0x2x0x0x0x2_harmonic/harmonic_extract/DFSET") as f:
    dfset = f.read().splitlines()
    

In [ ]:
alm_alm_opt = WorkflowFactory('alamode.alm_ALM_opt')
inputs = {"cwd": g_cwd, 
          "structure": g_superstructure, "dfset": List(list=dfset), 
         "norder": g_norder, "cutoff_radii": cutoff_radii, "nbody": nbody}
rt = submit(alm_alm_opt, **inputs)
wait_for_node_finished(rt)
rt

In [ ]:
rt.outputs.input_ANPHON

In [ ]:

raise

In [ ]:
from alamode_aiida.common.base import alamodeBaseCalcjob

In [ ]:
s = alamodeBaseCalcjob()
s.error.ERROR_OUTPUT_STDOUT_MISSING

In [ ]:
num_free = {}
len(num_free.keys())
